In [1]:
import pandas as pd
import glob
import utils
import json
from functools import reduce
from unidecode import unidecode

DATA = "./geojson/"
CSV = "./csv/"

In [2]:
files = glob.glob(f"{DATA}municipios*")

In [3]:
files

['./geojson/municipios_canarias.geojson',
 './geojson/municipios_peninsula.geojson']

In [4]:
with open(files[0], "r") as test:
    test2 = json.load(test)

In [5]:
with open(f"{DATA}/test.geojson", "w", encoding="utf-8") as fp:
    json.dump(test2, fp)

In [6]:
with open(f"{DATA}/test.geojson", "r") as test:
    test3 = json.load(test)
test2 == test3


True

In [7]:
with open(files[0], "r") as file:
    merged_json = json.load(file)

municipios = []
for file in files:
    with open(file, "r") as json_file:
        municipios.append(json.load(json_file)['features'])
        
merged_json['features'] = reduce(lambda a, b: a+b, municipios)

In [8]:
with open(f"{DATA}total_municipios.geojson", "w") as fp:
    json.dump(merged_json, fp)

In [9]:
df = pd.DataFrame.from_records([elem['properties'] for elem in merged_json['features']])
df.comunidad_id = df.comunidad_id.fillna(0) + df.comunida_id.fillna(0)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8219 entries, 0 to 8218
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   comunidad_name            8219 non-null   object 
 1   comunidad_normalizedName  8219 non-null   object 
 2   normalizedName            8219 non-null   object 
 3   name                      8219 non-null   object 
 4   provincia_name            8219 non-null   object 
 5   comunidad_id              8219 non-null   float64
 6   provincia_normalizedName  8219 non-null   object 
 7   provincia_id              8219 non-null   int64  
 8   id                        8219 non-null   int64  
 9   comunidad_id_indra        8219 non-null   int64  
 10  layer                     94 non-null     object 
 11  path                      94 non-null     object 
 12  fid                       8125 non-null   float64
 13  comunida_id               8125 non-null   float64
dtypes: float

In [11]:
df.normalizedName

0           agaete
1          aguimes
2          antigua
3         arrecife
4         artenara
           ...    
8214    vistabella
8215      zaida-la
8216      zaragoza
8217         zuera
8218          biel
Name: normalizedName, Length: 8219, dtype: object

In [12]:
votos = pd.read_csv(f"{CSV}02_201911_1_treat.csv")
votos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8131 entries, 0 to 8130
Data columns (total 83 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Unnamed: 0                 8131 non-null   int64 
 1   Nombre de Comunidad        8131 non-null   object
 2   Código de Provincia        8131 non-null   int64 
 3   Nombre de Provincia        8131 non-null   object
 4   Código de Municipio        8131 non-null   int64 
 5   Nombre de Municipio        8131 non-null   object
 6   Población                  8131 non-null   int64 
 7   Número de mesas            8131 non-null   int64 
 8   Total censo electoral      8131 non-null   int64 
 9   Total votantes             8131 non-null   int64 
 10  Votos válidos              8131 non-null   int64 
 11  Votos a candidaturas       8131 non-null   int64 
 12  Votos en blanco            8131 non-null   int64 
 13  Votos nulos                8131 non-null   int64 
 14  PSOE    

In [13]:
votos["Nombre Normalizado"] = votos["Nombre de Municipio"].apply(utils.rtve_norm)

In [14]:
"vilanova-i-la-geltru" in list(votos["Nombre Normalizado"])
"vilanova-i-la-geltru" in list(df["normalizedName"])

True

In [15]:
def municipio_check(muni):
    return(muni in list(votos['Nombre Normalizado']), muni in list(df['normalizedName']))

In [16]:
municipio_check("el-papiol")

(False, False)

In [17]:
for muni in list(df['normalizedName']):
    if not all(municipio_check(muni)):
        print(muni)

vilaflor
adsubia
alcoy-alcoi
alicante-alacant
benitachell-poble-nou-de-benitatxell-el
villajoyosa-vila-joiosa-la
campo-de-mirra-camp-de-mirra-el
elche-elx
fondo-de-les-neus-el-hondon-de-las-nieves
javea-xabia
jijona-xixona
lorcha-orxa-l
monovar-monover
pinos-el-pinoso
san-vicente-del-raspeig-sant-vicent-del-raspeig
torremanzanas-torre-de-les-macanes-la
Balanegra
banos-de-ebro-manueta
ribera-baja-erribera-beitia
ayala-aiara
campezo-kanpezu
elvillar-bilar
lanciego-lantziego
elburgo-burgelu
labastida-bastida
laudio-llodio
moreda-de-alava-moreda-araba
erriberagoitia-ribera-alta
salvatierra-agurain
san-millan-donemiliaga
valdegovia-gaubea
harana-valle-de-arana
villabuena-de-alava-eskuernaga
yecora-iekora
iruna-oka-iruna-de-oca
ametlla-del-valles-l
espunyola-l
estany-l
hospitalet-de-llobregat-l
santa-maria-de-corco
urduna-orduna
sopelana
karrantza-harana-valle-de-carranza
UNK
UNK
UNK
UNK
castrillo-matajudios
UNK
UNK
UNK
UNK
UNK
almazora-almassora
benasal
benicasim-benicassim
benlloch
borrian

In [23]:
test_df = df.merge(votos, how="left", left_on="normalizedName", right_on="Nombre Normalizado")
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8254 entries, 0 to 8253
Data columns (total 98 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   comunidad_name             8254 non-null   object 
 1   comunidad_normalizedName   8254 non-null   object 
 2   normalizedName             8254 non-null   object 
 3   name                       8254 non-null   object 
 4   provincia_name             8254 non-null   object 
 5   comunidad_id               8254 non-null   float64
 6   provincia_normalizedName   8254 non-null   object 
 7   provincia_id               8254 non-null   int64  
 8   id                         8254 non-null   int64  
 9   comunidad_id_indra         8254 non-null   int64  
 10  layer                      95 non-null     object 
 11  path                       95 non-null     object 
 12  fid                        8159 non-null   float64
 13  comunida_id                8159 non-null   float

In [27]:
test_json = json.loads(test_df.to_json(orient="records"))

In [31]:
def new_muni(muni_list, muni_name):
    """
    find city whose name matches
    """
    for muni in muni_list:
        if muni["normalizedName"] == muni_name:
            return muni
    return None

In [ ]:
def replace_muni(muni_list, muni_new_list):
    for muni in muni_list["features"]:
        
        muni_name = muni["properties"]["normalizedName"]
        
        replace_muni = new_muni(muni_new_list, muni_name)
        
        muni["properties"] = replace_muni if replace_muni else muni["

In [29]:
merged_json['features'][37]["properties"]

{'comunidad_name': 'Canarias',
 'comunidad_normalizedName': 'canarias',
 'normalizedName': 'adeje',
 'name': 'Adeje',
 'provincia_name': 'Santa Cruz de Tenerife',
 'comunidad_id': 5,
 'provincia_normalizedName': 'santa-cruz-de-tenerife',
 'provincia_id': 38,
 'id': 1,
 'comunidad_id_indra': 5,
 'layer': 'santa-cruz-de-tenerife provincias_peninsula',
 'path': 'E:/eclipse-rtve/rtve-app-elecciones/public/multimedia/mapas/provincias/santa-cruz-de-tenerife.geojson'}

In [30]:
test_json[37]

{'comunidad_name': 'Canarias',
 'comunidad_normalizedName': 'canarias',
 'normalizedName': 'arafo',
 'name': 'Arafo',
 'provincia_name': 'Santa Cruz de Tenerife',
 'comunidad_id': 5.0,
 'provincia_normalizedName': 'santa-cruz-de-tenerife',
 'provincia_id': 38,
 'id': 4,
 'comunidad_id_indra': 5,
 'layer': 'santa-cruz-de-tenerife provincias_peninsula',
 'path': 'E:/eclipse-rtve/rtve-app-elecciones/public/multimedia/mapas/provincias/santa-cruz-de-tenerife.geojson',
 'fid': None,
 'comunida_id': None,
 'Unnamed: 0': 1698.0,
 'Nombre de Comunidad': 'Canarias                      ',
 'Código de Provincia': 38.0,
 'Nombre de Provincia': 'Santa Cruz De Tenerife',
 'Código de Municipio': 4.0,
 'Nombre de Municipio': 'Arafo',
 'Población': 5562.0,
 'Número de mesas': 9.0,
 'Total censo electoral': 4536.0,
 'Total votantes': 2914.0,
 'Votos válidos': 2887.0,
 'Votos a candidaturas': 2859.0,
 'Votos en blanco': 28.0,
 'Votos nulos': 27.0,
 'PSOE': 907.0,
 'PP': 498.0,
 'VOX': 353.0,
 'PODEMOS-IU'